<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL2.0/blob/main/10_project_agg/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#10. Project Aggregation
**The projects table contains three columns: task_id, start_date, and end_date. The difference between end_date and start_date is 1 day for each row in the table. If task end dates are consecutive they are part of the same project. Projects do not overlap. Write a query to return the start and end dates of each project, and the number of days it took to complete. Order by ascending project duration, and ascending start date in the case of a tie.**

In [ ]:
!pip install polars
import pandas as pd
import numpy as  np
import polars as pl

In [ ]:
data = {'task_id'    : [1,2,3,4,5,6,7],
        'start_date' : ['01-OCT-20',
                        '02-OCT-20',
                        '03-OCT-20',
                        '13-OCT-20',
                        '14-OCT-20',
                        '28-OCT-20',
                        '30-OCT-20'],
         'end_date'  : ['02-OCT-20',
                        '03-OCT-20',
                        '04-OCT-20',
                        '14-OCT-20',
                        '15-OCT-20',
                        '29-OCT-20',
                        '31-OCT-20']
        }
pd_projects=pd.DataFrame(data)
pl_projects=pl.LazyFrame(data)
pd_projects['start_date']=(pd.to_datetime(pd_projects['start_date'],
                                          format="%d-%b-%y"
                              )
)
pd_projects['end_date']=(pd.to_datetime(pd_projects['end_date'],
                                        format="%d-%b-%y"
                            )
)
pl_projects=(pl_projects.with_columns(pl.col('start_date')
                                        .str
                                        .strptime(pl.Date,
                                                  format="%d-%b-%y"
                                         ),
                                      pl.col('end_date')
                                        .str
                                        .strptime(pl.Date,
                                                  format="%d-%b-%y"
                                         )
                         )
)

In [ ]:
print(f'----PANDAS----\nprojects table:\n{pd_projects}')

----PANDAS----
projects table:
   task_id start_date   end_date
0        1 2020-10-01 2020-10-02
1        2 2020-10-02 2020-10-03
2        3 2020-10-03 2020-10-04
3        4 2020-10-13 2020-10-14
4        5 2020-10-14 2020-10-15
5        6 2020-10-28 2020-10-29
6        7 2020-10-30 2020-10-31


In [ ]:
pd_df1=(pd_projects.sort_values(by=['task_id','start_date'])
                   .assign(project_id=lambda x:
                                      (x['start_date']!=x['end_date'].shift(1)).cumsum()
                    )
)[['task_id','project_id']]
print(f'Identifying unique projects:\n{pd_df1}')

Identifying unique projects:
   task_id  project_id
0        1           1
1        2           1
2        3           1
3        4           2
4        5           2
5        6           3
6        7           4


In [ ]:
pd_durations=(pd_projects.sort_values(by=['task_id','start_date'])
                         .assign(project_id=lambda x:
                                            (x['start_date']!=x['end_date'].shift(1))
                                                                           .cumsum()
                          )
                         .groupby('project_id',
                                  as_index=False
                          )
                         .agg(project_start=('start_date','first'),
                              project_end=('end_date','last')
                          )
                         .assign(duration=lambda x:
                                          (x['project_end']-x['project_start']).dt.days
                          )
                         .sort_values(by=['duration','project_start'])
)
print(f'Identifying durations:')
pd_durations

Identifying durations:


,project_id,project_start,project_end,duration
2,3,2020-10-28,2020-10-29,1
3,4,2020-10-30,2020-10-31,1
1,2,2020-10-13,2020-10-15,2
0,1,2020-10-01,2020-10-04,3


In [ ]:
sample=(pl_projects.head(5)
)
query1=sample.collect()
print(f'----POLARS----\nprojects table (SAMPLE):\n{query1}')

----POLARS----
projects table (SAMPLE):
shape: (5, 3)
┌─────────┬────────────┬────────────┐
│ task_id ┆ start_date ┆ end_date   │
│ ---     ┆ ---        ┆ ---        │
│ i64     ┆ date       ┆ date       │
╞═════════╪════════════╪════════════╡
│ 1       ┆ 2020-10-01 ┆ 2020-10-02 │
│ 2       ┆ 2020-10-02 ┆ 2020-10-03 │
│ 3       ┆ 2020-10-03 ┆ 2020-10-04 │
│ 4       ┆ 2020-10-13 ┆ 2020-10-14 │
│ 5       ┆ 2020-10-14 ┆ 2020-10-15 │
└─────────┴────────────┴────────────┘


In [ ]:
pl_df1=(sample.sort(by='start_date')
              .with_columns(project_id=pl.when(pl.col('start_date').eq(pl.col('end_date')
                                                                         .shift(1)
                                                                    )
                                          )
                                         .then(0)
                                         .otherwise(1)
                                         .cum_sum()
               )
              .select(pl.col('task_id'),
                      pl.col('project_id')
               )
).collect()
print(f'Identifying unique projects:\n{pl_df1}')

Identifying unique projects:
shape: (5, 2)
┌─────────┬────────────┐
│ task_id ┆ project_id │
│ ---     ┆ ---        │
│ i64     ┆ i32        │
╞═════════╪════════════╡
│ 1       ┆ 1          │
│ 2       ┆ 1          │
│ 3       ┆ 1          │
│ 4       ┆ 2          │
│ 5       ┆ 2          │
└─────────┴────────────┘


In [30]:
pl_durations=(pl_projects.sort(by='start_date')
                         .with_columns(project_id=pl.when(pl.col('start_date').eq(pl.col('end_date')
                                                                                    .shift(1)
                                                                               )
                                                     )
                                                    .then(0)
                                                    .otherwise(1)
                                                    .cum_sum()
                          )
                         .group_by('project_id')
                         .agg(project_start=pl.first('start_date'),
                              project_end=pl.last('end_date')
                          )
                         .with_columns(duration=(pl.col('project_end')-pl.col('project_start')).dt
                                                                                               .total_days()
                          )
                         .sort(by='duration')
).collect()
print(f'Durations of each unique project:')
pl_durations

Durations of each unique project:


project_id,project_start,project_end,duration
i32,date,date,i64
3,2020-10-28,2020-10-29,1
4,2020-10-30,2020-10-31,1
2,2020-10-13,2020-10-15,2
1,2020-10-01,2020-10-04,3
